위에 코드 실행한 후 밑에서 에러뜨면 사용
numpy 다운그레이드 해야 import gluonnlp as nlp에서 오류 발생하지 않음

In [1]:
!pip uninstall numpy
!pip install numpy==1.23.1

Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.10
    /usr/local/lib/python3.10/dist-packages/numpy-1.25.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-5007b62f.3.23.dev.so
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; pyth

In [1]:
!pip install gluonnlp pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661750 sha256=ae9e2870ec88879887bceff1b5714f93ce09fff147f368900e6ea84f05033405
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [2]:
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

=====여기서부터 실행======

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-qmaz49z0/kobert-tokenizer_54038c524ea24333b74b28aef3ca7374
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-qmaz49z0/kobert-tokenizer_54038c524ea24333b74b28aef3ca7374
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=5c99f9c6c5417ca69e926b4548542f0df4c869b8481bc92bfcc4f9a30e49dfbb
  Stored in directory: /tmp/pip-ephem-wheel-cache-b9j4jw1i/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [15]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx')

In [ ]:
chatbot_data.fillna('',inplace=True)

In [ ]:
chatbot_data.isnull().values.any()

False

In [ ]:
# 여러개의 열 하나로 합치기
def combine_columns(row):
  return ' '.join(row.astype(str))

chatbot_data['Sentence1'] = chatbot_data[['사람문장1','시스템문장1']].apply(combine_columns,axis=1)

print(chatbot_data.columns)

Index(['Unnamed: 0', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
       '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3', 'Sentence1'],
      dtype='object')


In [ ]:
chatbot_data['감정_대분류'].unique()

array(['분노', '기쁨', '불안', '당황', '슬픔', '상처'], dtype=object)

In [22]:
chatbot_data.rename(columns={'감정_대분류':'Emotion'},inplace=True)

In [16]:
chatbot_data1 = pd.read_excel('/content/drive/MyDrive/한국어 감정 정보가 포함된 단발성 대화 데이터셋/한국어_단발성_대화_데이터셋.xlsx')

In [17]:
chatbot_data1['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

In [18]:
# 조건에 따라 특정 행 제거하기
chatbot_data1 = chatbot_data1[~chatbot_data1['Emotion'].isin(['중립', '혐오'])]

In [19]:
chatbot_data1.loc[(chatbot_data1['Emotion']=='공포'),'Emotion'] = 0
chatbot_data1.loc[(chatbot_data1['Emotion']=='놀람'),'Emotion'] = 1
chatbot_data1.loc[(chatbot_data1['Emotion']=='분노'),'Emotion'] = 2
chatbot_data1.loc[(chatbot_data1['Emotion']=='슬픔'),'Emotion'] = 3
chatbot_data1.loc[(chatbot_data1['Emotion']=='행복'),'Emotion'] = 4

In [23]:
chatbot_data.loc[(chatbot_data['Emotion']=='불안'),'Emotion'] = 0
chatbot_data.loc[(chatbot_data['Emotion']=='당황'),'Emotion'] = 1
chatbot_data.loc[(chatbot_data['Emotion']=='분노'),'Emotion'] = 2
# 슬픔, 상처를 하나의 라벨로 통합
chatbot_data.loc[(chatbot_data['Emotion']=='슬픔'),'Emotion'] = 3
chatbot_data.loc[(chatbot_data['Emotion']=='상처'),'Emotion'] = 3
chatbot_data.loc[(chatbot_data['Emotion']=='기쁨'),'Emotion'] = 4

sentence 내에 공백이 여러개 있을 수 있으므로 하나로 치환

In [ ]:
chatbot_data['Sentence1'] = chatbot_data['Sentence1'].str.replace(r'\s+', ' ', regex=True)

In [ ]:
chatbot_data.head()

,Unnamed: 0,연령,성별,상황키워드,신체질환,Emotion,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3,Sentence1
0,1,청년,여성,"진로,취업,직장",해당없음,2,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,,,일은 왜 해도 해도 끝이 없을까? 화가 난다. 많이 힘드시겠어요. 주위에 의논할 상...
1,2,청년,여성,"진로,취업,직장",해당없음,2,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,,,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,3,청년,여성,"진로,취업,직장",해당없음,2,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,,,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,4,청년,여성,"진로,취업,직장",해당없음,2,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,,,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,5,청년,여성,"진로,취업,직장",해당없음,2,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,,,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 무시하는 것 같...


In [ ]:
chatbot_data['Emotion'][0]


2

In [24]:
data_list = []
for q, label in zip(chatbot_data['사람문장1'], chatbot_data['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))
  data_list.append(data)
'''
for q, label in zip(chatbot_data1['Sentence'], chatbot_data1['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))
  data_list.append(data)
'''

"\nfor q, label in zip(chatbot_data1['Sentence'], chatbot_data1['Emotion']):\n  data = []\n  data.append(q)\n  data.append(str(label))\n  data_list.append(data)\n"

In [ ]:
print(len(data_list))
print(data_list[0])
print(data_list[-1])

51630
['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '2']
['남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져.', '3']


In [28]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2)

print(len(dataset_train))
print(dataset_train[100])
print(len(dataset_test))

41304
['갱년기가 오니 몸도 마음도 평소 같지 않고 힘들어. 잘 넘겨야겠어.', '2']
10326


gluonnlp의 BERTSentenceTransform이 KoBERTTokenizer와 호환되지 않으므로 직접 tokenizer를 사용하여 토큰화 과정을 처리

In [12]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [13]:
#tokenizer = nlp.data.BERTTokenizer(tokenizer=tokenizer,vocab=vocab,lower=True)
tokenizer = tokenizer.tokenize
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair, vocab):
        self.transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair, vocab=vocab)
        self.sentences = [self.transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return self.sentences[i] + (self.labels[i],)

    def __len__(self):
        return len(self.labels)

In [29]:
data_train = BERTDataset(dataset_train, 0, 1,tokenizer, max_len, True, False,vocab)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, max_len, True, False,vocab)

In [ ]:
data_test[0]
data_test[4]

(array([   2, 4571, 7782, 7160,  806, 4566, 6116,  517, 7857, 5850, 2110,
        2689, 7123, 5101, 6669, 5839,   54,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [30]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=2, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=2, shuffle=True)

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [6]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    checkpoint = {
    'epoch': e,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'accuracy':test_acc / (batch_id+1),
    }
    # 저장 경로 설정
    save_path = '/content/drive/MyDrive/TextToEmotion1(KoBERT)_epoch{}.pt'.format(e)
    # 딕셔너리를 파일로 저장
    torch.save(checkpoint, save_path)

<ipython-input-40-dacb255d50be>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/20652 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7767874002456665 train acc 0.5
epoch 1 batch id 201 loss 1.7206270694732666 train acc 0.13681592039800994
epoch 1 batch id 401 loss 2.0058681964874268 train acc 0.1433915211970075
epoch 1 batch id 601 loss 1.9970030784606934 train acc 0.16638935108153077
epoch 1 batch id 801 loss 2.043989896774292 train acc 0.20786516853932585
epoch 1 batch id 1001 loss 1.6947522163391113 train acc 0.23026973026973027
epoch 1 batch id 1201 loss 1.9263396263122559 train acc 0.25104079933388845
epoch 1 batch id 1401 loss 1.9389755725860596 train acc 0.2651677373304782
epoch 1 batch id 1601 loss 1.4686956405639648 train acc 0.27514053716427234
epoch 1 batch id 1801 loss 1.018916130065918 train acc 0.2787340366463076
epoch 1 batch id 2001 loss 1.542700171470642 train acc 0.2863568215892054
epoch 1 batch id 2201 loss 2.0385777950286865 train acc 0.29123125851885506
epoch 1 batch id 2401 loss 1.588494896888733 train acc 0.2940441482715535
epoch 1 batch id 2601 loss 1.114916086196899

<ipython-input-40-dacb255d50be>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/5163 [00:00<?, ?it/s]

...

epoch 2 batch id 1 loss 1.279721975326538 train acc 0.5
epoch 2 batch id 201 loss 1.9568145275115967 train acc 0.3407960199004975
epoch 2 batch id 401 loss 2.2304370403289795 train acc 0.3553615960099751
epoch 2 batch id 601 loss 1.3770616054534912 train acc 0.3502495840266223
epoch 2 batch id 801 loss 1.3522920608520508 train acc 0.3620474406991261
epoch 2 batch id 1001 loss 1.8366117477416992 train acc 0.35714285714285715
epoch 2 batch id 1201 loss 1.8824162483215332 train acc 0.35512073272273104
epoch 2 batch id 1401 loss 2.0499939918518066 train acc 0.3600999286224126
epoch 2 batch id 1601 loss 1.5911792516708374 train acc 0.3588382261086821
epoch 2 batch id 1801 loss 1.7626270055770874 train acc 0.3586896168795114
epoch 2 batch id 2001 loss 2.145561695098877 train acc 0.3588205897051474
epoch 2 batch id 2201 loss 1.6730899810791016 train acc 0.35529304861426625
epoch 2 batch id 2401 loss 1.4285595417022705 train acc 0.35443565181174513
epoch 2 batch id 2601 loss 1.8637909889221191

  0%|          | 0/5163 [00:00<?, ?it/s]

epoch 2 test acc 0.35657563432113115


  0%|          | 0/20652 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 1.7979209423065186 train acc 0.0
epoch 3 batch id 201 loss 2.2745890617370605 train acc 0.35323383084577115
epoch 3 batch id 401 loss 1.3364064693450928 train acc 0.3491271820448878
epoch 3 batch id 601 loss 1.721697449684143 train acc 0.34359400998336104
epoch 3 batch id 801 loss 1.8550716638565063 train acc 0.3545568039950062
epoch 3 batch id 1001 loss 1.4878321886062622 train acc 0.35764235764235763
epoch 3 batch id 1201 loss 1.386565923690796 train acc 0.36219816819317235
epoch 3 batch id 1401 loss 1.416008472442627 train acc 0.3579586009992862
epoch 3 batch id 1601 loss 1.7557440996170044 train acc 0.35633978763272955
epoch 3 batch id 1801 loss 1.5400011539459229 train acc 0.3561910049972238
epoch 3 batch id 2001 loss 1.8492555618286133 train acc 0.3575712143928036
epoch 3 batch id 2201 loss 1.3156954050064087 train acc 0.3566560654248069
epoch 3 batch id 2401 loss 1.5376558303833008 train acc 0.3548521449396085
epoch 3 batch id 2601 loss 1.4116132259368896

In [35]:
saved_epoch=0
checkpoint = torch.load('/content/drive/MyDrive/TextToEmotion1(KoBERT)_epoch5.pt'.format(saved_epoch))
# 모델과 옵티마이저의 상태 복원
model.load_state_dict(checkpoint['model_state_dict'])
accuracy = checkpoint['accuracy']
print(accuracy)

0.8949948400412797
